# Tahu lah apa ini?

In [2]:
# Path to google drive folder
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002. Laporan-eksperimen
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024/

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest, VotingClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from pyod.models.hbos import HBOS
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, OutlierMixin
from scipy.spatial.distance import mahalanobis

In [4]:
# Load the dataset
# file_path = '01.Dataset-Rofiqul/agriculture_dataset.csv'
# file_path = '01.Dataset-Rofiqul/Ujicoba3-29052024/agriculture_dataset.csv'
file_path = 'dataset/agriculture_dataset.csv'
# file_path = 'dataset/09_05_1_AgricultureOutliersResult_30052024.csv'
data = pd.read_csv(file_path)


In [5]:

# Display the first few rows of the dataset for review
# data.head(),data.info(),data.describe()
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,PP7,AirT,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,0.00,-2.00,276,241,0.666508,11.046340,22.940812,62.500000,637.500000,1.174072
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,0.00,-2.40,277,242,0.640608,11.008087,22.959578,62.500000,637.500000,1.174072
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,8.64,0.30,285,250,0.728085,10.831669,23.221928,62.500000,637.500000,1.174072
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,8.13,-3.80,286,251,0.686872,10.849792,23.271978,62.500000,637.500000,1.174072
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,8.39,17.60,312,277,0.716221,10.204748,24.206855,62.500000,637.500000,1.174072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,8.13,10.81,167,167,0.390203,17.061793,5.803681,128.333333,491.666667,1.806000
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,8.13,5.64,197,197,0.317040,5.624730,5.429873,162.500000,490.375000,1.496400
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,8.13,5.64,197,197,0.341428,3.504328,7.239000,183.250000,432.875000,1.186800
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,8.13,5.64,197,197,0.365815,3.636931,8.434294,169.833333,418.833333,1.823200


## Detect Nilai NaN

In [6]:

missing_values_count = data.isna().sum()
missing_values_count

Date            0
Year            0
Experiment      0
DataUse         0
Replication     0
Month           0
Vegetation      0
VegType         0
N2O             0
N_rate          0
PP2             0
PP7             0
AirT            0
DAF_TD          0
DAF_SD          0
WFPS25cm       52
NH4            76
NO3            30
Clay            0
Sand            0
SOM             0
dtype: int64

## Impute NaN or missing value with Mean

In [7]:
# Impute missing values with the mean of the respective columns
data['WFPS25cm'].fillna(data['WFPS25cm'].mean(), inplace=True)
data['NH4'].fillna(data['NH4'].mean(), inplace=True)
data['NO3'].fillna(data['NO3'].mean(), inplace=True)

# Verify that there are no missing values left
missing_values_after_imputation = data.isna().sum()
missing_values_after_imputation

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16824\190507584.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['WFPS25cm'].fillna(data['WFPS25cm'].mean(), inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16824\190507584.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

Date           0
Year           0
Experiment     0
DataUse        0
Replication    0
Month          0
Vegetation     0
VegType        0
N2O            0
N_rate         0
PP2            0
PP7            0
AirT           0
DAF_TD         0
DAF_SD         0
WFPS25cm       0
NH4            0
NO3            0
Clay           0
Sand           0
SOM            0
dtype: int64

In [8]:
# Save the updated data with outlier detection results
data.to_csv('dataset/09_05_2_0_AgricultureNoMissingValueResult.csv', index=False)

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the data
# file_path = 'path_to_your_csv_file.csv'
# data = pd.read_csv(file_path)
X = data[['N2O']]

# Initialize the outlier detection models
isolation_forest = IsolationForest(contamination=0.1, random_state=42)
lof = LocalOutlierFactor(contamination=0.1, novelty=True)
one_class_svm = OneClassSVM(nu=0.1)
elliptic_envelope = EllipticEnvelope(contamination=0.1)
hbos = HBOS()
dbscan = DBSCAN(eps=0.5, min_samples=5)
# Apply OPTICS clustering to detect outliers
optics = OPTICS(eps=0.5, min_samples=5)


# Get the outlier indices from OPTICS
optics_outliers = np.where(optics.labels_ == -1)[0]
# Fit the models and predict outliers
data['if_outlier'] = isolation_forest.fit_predict(X)
lof.fit(X)
data['lof_outlier'] = lof.predict(X)
data['ocsvm_outlier'] = one_class_svm.fit_predict(X)
data['elliptic_outlier'] = elliptic_envelope.fit_predict(X)
data['hbos_outlier'] = hbos.fit_predict(X)
data['dbscan_outlier'] = dbscan.fit_predict(X)
data['optics_outlier'] = optics.fit(X.values.reshape(-1, 1))

# Calculate Mahalanobis Distance and detect outliers
cov_matrix = X.cov().to_numpy()
mean_distr = X.mean().to_numpy()
inv_cov_matrix = np.linalg.inv(cov_matrix)

def mahalanobis_distance(x, mean, inv_cov_matrix):
    return mahalanobis(x, mean, inv_cov_matrix)

data['mahalanobis_distance'] = X.apply(lambda x: mahalanobis_distance(x, mean_distr, inv_cov_matrix), axis=1)
threshold = data['mahalanobis_distance'].quantile(0.95)
data['mahalanobis_outlier'] = (data['mahalanobis_distance'] > threshold).astype(int)

# Generate synthetic ground truth labels for evaluation purposes
# Creating a synthetic ground truth based on majority voting of outliers
data['majority_voting'] = data.apply(lambda row: 1 if row[['if_outlier', 'lof_outlier', 'ocsvm_outlier', 'elliptic_outlier', 'hbos_outlier', 'dbscan_outlier', 'mahalanobis_outlier']].mode()[0] == 1 else -1, axis=1)

# Append the outlier results to the original data
# data = pd.concat([data, data[['IF_outlier', 'LOF_Outlier', 'OCSVM_Outlier', 'Elliptic_Outlier', 'HBOS_Outlier', 'DBSCAN_Outlier', 'mahalanobis_Outlier', 'Majority_Voting']]], axis=1)

# Save the updated data to the CSV file
data.to_csv('dataset/09_05_1_AgricultureOutliersResult_30052024.csv', index=False)

# Display the updated dataframe with outlier detection results
# print(data.head())
# data

NameError: name 'OPTICS' is not defined

In [ ]:
data.info()

In [ ]:
# Creating a synthetic ground truth based on majority voting of outliers
# data['majority'] = data.apply(lambda row: 1 if row[['if_outlier','lof_outlier','ocsvm_outlier','elliptic_outlier','hbos_outlier','dbscan_outlier','mahalanobis_outlier']].mode()[0] == 1 else -1, axis=1)
# Convert outlier labels to binary (1 for outliers, 0 for inliers) for evaluation metrics
y_true = (data['majority_voting'] == -1).astype(int)

In [ ]:
data

In [ ]:


# Voting-based with averaging (soft voting)
data['average_voting'] = data[[f'{name}_outlier' for name, _ in models]].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)


In [ ]:
data

In [ ]:
# Weighted voting
weights = {
    'if': 1,
    'lof': 1,
    'ocsvm': 1,
    'elliptic': 1,
    'hbos': 1,
    'dbscan': 1,
    'mahalanobis': 1
}
data['weighted_voting'] = data[[f'{name}_outlier' for name, _ in models]].apply(
    lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)


In [ ]:
data

In [ ]:
# Save the updated data with outlier detection results
data.to_csv('dataset/09_05_2_1_AgricultureOutliersVotingBasedResult.csv', index=False)

In [ ]:
data.info()

In [ ]:
# Generate statistics for each detection method and voting-based result
methods = ['if_outlier', 'lof_outlier', 'ocsvm_outlier', 'elliptic_outlier', 'hbos_outlier', 'dbscan_outlier', 'mahalanobis_outlier', 'average_voting', 'weighted_voting', 'majority_voting']

statistics = {}

for method in methods:
    counts = data[method].value_counts()
    stats = {
        'Inliers': counts.get(1, 0),
        'Outliers': counts.get(-1, 0),
        'Total': counts.sum()
    }
    statistics[method] = stats

# Convert the statistics to a DataFrame for better visualization
statistics_df = pd.DataFrame(statistics).T
statistics_df.to_csv('dataset/09_05_2_2_AgricultureOutliersVotingBasedStatsSummary.csv', index=False)
statistics_df